In [ ]:
from vectorrvnn.utils import *
from vectorrvnn.trainutils import *
from vectorrvnn.network import *
import matplotlib.pyplot as plt
import torch
from torch import nn
from torchvision.ops import *
from torchvision.models import *
import svgpathtools as svg

opts = Options().parse(testing=[
    '--checkpoints_dir', '../results',
    '--dataroot', '../data/All',
    '--name', 'test',
    '--samplercls', 'DiscriminativeSampler',
    '--phase', 'test',
    '--backbone', 'resnet18',
])
roinet = RoIAlignNet(opts)

In [ ]:
def visGradient (model, bbox, input) :
    model = model.cuda()
    input.requires_grad = True
    output = model(input, bbox.cuda())
    output.sum().backward()
    grad = np.sqrt((input.grad * input.grad).squeeze().sum(0).detach().cpu().numpy())
    pctEffected = (grad > 0).sum() / (input.nelement() / 3)
    print("% effected elements: ", pctEffected)
    plt.imshow(np.log(grad), cmap='bwr')
    plt.colorbar()
    plt.show()

In [ ]:
boxes = [
    [0., 0., 1., 1.],
    [0.5, 0.5, 0.6, 0.6],
    [0.2, 0.2, 0.3, 0.3],
    [0.1, 0.1, 0.101, 0.101],
    [0.0, 0.0, 0.001, 0.001],
    [0.5, 0.5, 0.501, 0.501],
    [0.6, 0.6, 0.7, 0.602],
    [0.1, 0.1, 0.4, 0.4],
    [0.2, 0.2, 0.8, 0.8],
    [0.5, 0.5, 0.9, 0.9],
]

doc = svg.Document('./Emojis/Test/1F682.svg')
plt.imshow(rasterize(doc, 256, 256))
plt.show()

for box in boxes : 
    print('bbox:', box)
    bbox = torch.tensor([box])
    input = getTransform(opts)(rasterize(doc, 256, 256)).unsqueeze(0).cuda()
    visGradient(roinet, bbox, input)